# Network Science Final Project

## 1. Data preprocessing

In [30]:
import pandas as pd
import numpy as np
import networkx as nx
import re
import warnings
warnings.simplefilter('ignore')

In [2]:
# import dat files as dataframes
df_routes = pd.read_table(r"data\openFlights_airline\routes.dat.txt", sep=",", header=None )
df_airline = pd.read_table(r"data\openFlights_airline\airlines.dat.txt", sep=",", header=None )
df_planes = pd.read_table(r"data\openFlights_airline\planes.dat.txt", sep=",", header=None )
df_airports = pd.read_table(r"data\openFlights_airline\airports.dat.txt", sep=",", header=None )
print(f"df_routes is in the shape of {df_routes.shape}")
df_routes.head(5)

df_routes is in the shape of (67663, 9)


,0,1,2,3,4,5,6,7,8
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2


In [3]:
# set the correct column names according to the information from the data source
df_routes.columns =['code','ID','Source_airport','Source_airport_ID','Des_airport','Des_airport_ID','Codeshare','Stops','Equipment']
df_airline.columns =['Airline ID','Name','Alias','IATA','ICAO','Callsign','Country','Active']
df_planes.columns =['name','iso_code','dafif_code']
df_airports.columns =['Airport ID','Name','City','Country','IATA','ICAO','Latitude','Longitude','Altitude','Timezone','DST','Tz_database_time','Type','Source']
df_routes.head(3)

,code,ID,Source_airport,Source_airport_ID,Des_airport,Des_airport_ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2


In [7]:
df_routes_missing = df_routes[df_routes.eq(r"\N").any(1)]
print(f"df_routes_missing is in the shape of {df_routes_missing.shape}")
df_routes_missing 

df_routes_missing is in the shape of (898, 8)


C:\Users\Guan\AppData\Local\Temp\ipykernel_19220\1819816128.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_routes_missing = df_routes[df_routes.eq(r"\N").any(1)]


,code,ID,Source_airport,Source_airport_ID,Des_airport,Des_airport_ID,Stops,Equipment
7,2B,410,DME,4029,TGK,\N,0,CR2
38,2B,410,TGK,\N,DME,4029,0,CR2
48,2G,1654,IKT,2937,KCK,\N,0,AN4
54,2G,1654,KCK,\N,IKT,2937,0,AN4
312,3H,\N,AKV,5506,YIK,5504,0,DH8
...,...,...,...,...,...,...,...,...
67469,ZI,21,MLH,\N,CZL,221,0,320
67470,ZI,21,MLH,\N,ORN,231,0,319
67471,ZI,21,MLH,\N,QSF,6492,0,320
67483,ZI,21,ORN,231,MLH,\N,0,319


In [8]:
df_routes_id_missing = df_routes[((df_routes['Source_airport_ID']==r'\N')|(df_routes['Des_airport_ID']==r'\N'))]
df_routes_id_missing.shape

(423, 8)

In [6]:
# check the missing or wrong values in the airport data
if 'Codeshare' in df_routes:
    df_routes = df_routes.drop('Codeshare', axis=1) 
    
iata_ico = r'[A-Z]{3,4}'
airportId = r'[0-9]{1,4}'

df_routes_missing= df_routes[((~df_routes['Source_airport'].str.match(iata_ico))|(~df_routes['Des_airport'].str.match(iata_ico)))]

print(f"df_routes_missing is in the shape of {df_routes_missing.shape}")
df_routes_missing



df_routes_missing is in the shape of (0, 8)


,code,ID,Source_airport,Source_airport_ID,Des_airport,Des_airport_ID,Stops,Equipment


As shown above, there are 898 entries with missing values and among them, 423 entries has missing values in the column Source_airport_ID and Des_airport_ID, we filled this missing IDs with their airport code. Since there are no missing values in Source_airport and Des_airport, we will use them to build the network.

In [28]:
def replace_missing_IDs(df):
    for index, row in df.iterrows(): 
        if row['Source_airport_ID']==r"\N":
            df['Source_airport_ID'][index]=row['Source_airport']+"_IDs"
        if row['Des_airport_ID']==r"\N":
            df['Des_airport_ID'][index]=row['Des_airport']+"_IDs"

In [31]:
replace_missing_IDs(df_routes)
df_routes_id_missing = df_routes[((df_routes['Source_airport_ID']==r'\N')|(df_routes['Des_airport_ID']==r'\N'))]
df_routes_id_missing.shape

(0, 8)

## 2. Graph building

In [36]:
g_routes = nx.from_pandas_edgelist(df_routes, source='Source_airport', target = 'Des_airport',edge_attr=True,
                                   create_using=nx.DiGraph())
g_routes

In [38]:
source_airports = df_routes["Source_airport_ID"].unique()
des_airports = df_routes["Des_airport_ID"].unique()
print(f'There are {len(source_airports)} airports as source airports')
print(f'There are {len(des_airports)} airports as destination airports')

There are 3409 airports as source airports
There are 3418 airports as destination airports


In [39]:
gnode=set(g_routes.nodes())
snode=set(df_routes["Source_airport"].unique())
dnode=set(df_routes["Des_airport"].unique())
print(f'The airports only serve as a source airport: {snode-dnode}')
print(f'The airports only serve as a destination airport: {dnode-snode}')

The airports only serve as a source airport: {'PTJ', 'IUE', 'STZ', 'VDA', 'SXX', 'LJA', 'MSW'}
The airports only serve as a destination airport: {'BVS', 'DLZ', 'MTE', 'KPR', 'TUA', 'UII', 'KYK', 'FMI', 'ORX', 'CZJ', 'SPI', 'KZB', 'BSS', 'KZI', 'CMP', 'QFX'}
